In [113]:
from numpy import exp
from numpy import log
from scipy.special import factorial
from scipy.stats import norm
import matplotlib.pyplot as plt

In [114]:
norm.pdf(0)

0.3989422804014327

In [115]:
norm.cdf(0)

0.5

In [260]:
import numpy as np
class PoissonRegression:
    
    def __init__(self, y, X, β):
        self.X, self.y, self.β = X, y, β 
        self.n, self.k = X.shape
    
    def μ(self):
        return np.exp(self.X @ self.β.T)
    
    def logL(self): 
        y = self.y
        μ = self.μ()
        return np.sum(y * np.log(μ) - μ - np.log(factorial(y)))

    def G(self):
        μ = self.μ()
        return ((self.y - μ) @ self.X).reshape(self.k, 1)
    
    def H(self):
        X = self.X
        μ = self.μ()
        return -(μ * X.T @ X)

In [265]:
X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])
y = np.array([1, 0, 1, 1, 0])
# Take a guess at initial βs 
init_β = np.array([0.1, 0.1, 0.1])

In [267]:
# Create an object with Poisson model values
poi = PoissonRegression(y, X, β=init_β)

In [268]:
poi.H()

array([[  -9.76227711,  -30.22868241,  -26.201177  ],
       [ -30.22868241, -113.24794295,  -70.51316785],
       [ -26.201177  ,  -70.51316785,  -89.08291046]])

In [269]:
poi.y

array([1, 0, 1, 1, 0])

In [121]:
X[0,]

array([1, 2, 5])

In [270]:
poi.G()

array([[ -6.76227711],
       [-19.22868241],
       [-17.201177  ]])

In [271]:
poi.n

5

In [272]:
d = np.zeros((1,3))

for i in range(poi.n):
    d = d + (poi.y[i]-poi.μ()[i]) * X[i,]
d.reshape(poi.k, 1)

array([[ -6.76227711],
       [-19.22868241],
       [-17.201177  ]])

In [127]:
poi.y

array([1, 0, 1, 1, 0])

In [128]:
poi.μ()

array([2.22554093, 1.64872127, 2.01375271, 2.22554093, 1.64872127])

In [129]:
list(init_β)

[0.1, 0.1, 0.1]

In [130]:
def newton_raphson(model, tol=1e-3, max_iter=1000, display=True):

    i = 0
    error = 100  # Initial error value

    # Print header of output
    if display:
        header = f'{"Iteration_k":<13}{"Log-likelihood":<16}{"θ":<60}'
        print(header)
        print("-" * len(header))

    # While loop runs while any value in error is greater
    # than the tolerance until max iterations are reached
    while np.any(error > tol) and i < max_iter:
        H, G = model.H(), model.G()
        β_new = model.β - (np.linalg.inv(H) @ G).T
        error = β_new - model.β
        model.β = β_new.flatten()

        # Print iterations
        if display:
            β_list = [f'{t:.3}' for t in list(model.β)]
            update = f'{i:<13}{model.logL():<16.8}{β_list}'
            print(update)

        i += 1

    print(f'Number of iterations: {i}')
    print(f'β_hat = {model.β}')

    return model.β

In [131]:
X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])

y = np.array([1, 0, 1, 1, 0])

# Take a guess at initial βs
init_β = np.array([0.1, 0.1, 0.1])

# Create an object with Poisson model values
poi = PoissonRegression(y, X, β=init_β)

# Use newton_raphson to find the MLE
β_hat = newton_raphson(poi, display=True)

Iteration_k  Log-likelihood  θ                                                           
-----------------------------------------------------------------------------------------
0            -4.3447622      ['-1.49', '0.265', '0.244']
1            -3.5742413      ['-3.38', '0.528', '0.474']
2            -3.3999526      ['-5.06', '0.782', '0.702']
3            -3.3788646      ['-5.92', '0.909', '0.82']
4            -3.3783559      ['-6.07', '0.933', '0.843']
5            -3.3783555      ['-6.08', '0.933', '0.843']
Number of iterations: 6
β_hat = [-6.07848205  0.93340226  0.84329625]


In [132]:
from statsmodels.api import Poisson
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])

y = np.array([1, 0, 1, 1, 0])

stats_poisson = Poisson(y, X).fit()
print(stats_poisson.summary())

Optimization terminated successfully.
         Current function value: 0.675671
         Iterations 7
                          Poisson Regression Results                          
Dep. Variable:                      y   No. Observations:                    5
Model:                        Poisson   Df Residuals:                        2
Method:                           MLE   Df Model:                            2
Date:                Sun, 10 Jun 2018   Pseudo R-squ.:                  0.2546
Time:                        17:46:55   Log-Likelihood:                -3.3784
converged:                       True   LL-Null:                       -4.5325
                                        LLR p-value:                    0.3153
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.0785      5.279     -1.151      0.250     -16.425       4.268
x1             0.9334      0.

In [27]:
import pandas as pd
pd.options.display.max_columns = 10

# Load in data and view
df = pd.read_stata('https://github.com/QuantEcon/QuantEcon.lectures.code/raw/master/mle/fp.dta')
df.head()

,country,ccode,year,cyear,numbil,...,topint08,rintr,noyrs,roflaw,nrrents
0,United States,2.0,1990.0,21990.0,NaN,...,39.799999,4.988405,20.0,1.61,NaN
1,United States,2.0,1991.0,21991.0,NaN,...,39.799999,4.988405,20.0,1.61,NaN
2,United States,2.0,1992.0,21992.0,NaN,...,39.799999,4.988405,20.0,1.61,NaN
3,United States,2.0,1993.0,21993.0,NaN,...,39.799999,4.988405,20.0,1.61,NaN
4,United States,2.0,1994.0,21994.0,NaN,...,39.799999,4.988405,20.0,1.61,NaN


In [28]:
# Keep only year 2008
df = df[df['year'] == 2008]

In [29]:
# Add a constant
df['const'] = 1

# Variable sets
reg1 = ['const', 'lngdppc', 'lnpop', 'gattwto08']
reg2 = ['const', 'lngdppc', 'lnpop',
        'gattwto08', 'lnmcap08', 'rintr', 'topint08']
reg3 = ['const', 'lngdppc', 'lnpop', 'gattwto08', 'lnmcap08',
        'rintr', 'topint08', 'nrrents', 'roflaw']

In [30]:
import statsmodels.api as sm

# Specify model
poisson_reg = sm.Poisson(df[['numbil0']], df[reg1],
                         missing='drop').fit(cov_type='HC0')
print(poisson_reg.summary())

Optimization terminated successfully.
         Current function value: 2.226090
         Iterations 9
                          Poisson Regression Results                          
Dep. Variable:                numbil0   No. Observations:                  197
Model:                        Poisson   Df Residuals:                      193
Method:                           MLE   Df Model:                            3
Date:                Sun, 10 Jun 2018   Pseudo R-squ.:                  0.8574
Time:                        11:06:40   Log-Likelihood:                -438.54
converged:                       True   LL-Null:                       -3074.7
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -29.0495      2.578    -11.268      0.000     -34.103     -23.997
lngdppc        1.0839      0.

In [353]:
class ProbitRegr:
    
    def __init__(self, y, X, β):
        self.X, self.y, self.β = X, y, β 
        self.N, self.k = X.shape
    
    def μ(self,i, q=1):
        return norm.cdf(q*self.X[i,] @ self.β.T)
    
    def f(self,i, q=1):
        return norm.pdf(q*self.X[i,] @ self.β.T)
    
    def λ(self,i):
        y = self.y
        q = 2*y[i]-1
        λ = self.f(i, q)/self.μ(i, q)*q
        return λ

    def logL(self): 
        y = self.y
        L = 0
        for i in range(self.N):
            L = L + y[i]*log(self.μ(i)) + (1-y[i])*log(1-self.μ(i))
        return L
    
    def G(self):
        y = self.y
        X = self.X
        G = 0
        for i in range(self.N):
            G = G + self.f(i)/((1-self.μ(i))*(self.μ(i)))*(y[i]-self.μ(i))*X[i,]
        return G.reshape(self.k, 1)
    
    def H(self):
        X = self.X
        β = self.β
        H = 0
        for i in range(self.N):
            X_i = X[i,].reshape(self.k,1)
            H = H - self.λ(i)*(X[i,]@self.β + self.λ(i))*(X_i@X_i.T)
        return H

In [357]:
X = np.array([[1, 2, 4],
              [1, 1, 1],
              [1, 4, 3],
              [1, 5, 6],
              [1, 3, 5]])

y = np.array([1, 0, 1, 1, 0])

# Take a guess at initial βs
β = np.array([0.1, 0.1, 0.1])

# Create an object with Probit model values
prob = ProbitRegr(y, X, β)

# Use newton_raphson to find the MLE
β_hat = newton_raphson(prob, display=True)

Iteration_k  Log-likelihood  θ                                                           
-----------------------------------------------------------------------------------------
0            -2.3796884      ['-1.34', '0.775', '-0.157']
1            -2.3687526      ['-1.53', '0.775', '-0.0981']
2            -2.3687294      ['-1.55', '0.778', '-0.0971']
3            -2.3687294      ['-1.55', '0.778', '-0.0971']
Number of iterations: 4
β_hat = [-1.54625858  0.77778952 -0.09709757]


In [250]:
from statsmodels.discrete.discrete_model import Probit

In [337]:
print(Probit(y, X).fit().summary())

Optimization terminated successfully.
         Current function value: 0.473746
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:                    5
Model:                         Probit   Df Residuals:                        2
Method:                           MLE   Df Model:                            2
Date:                Mon, 11 Jun 2018   Pseudo R-squ.:                  0.2961
Time:                        10:16:05   Log-Likelihood:                -2.3687
converged:                       True   LL-Null:                       -3.3651
                                        LLR p-value:                    0.3692
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5463      1.866     -0.829      0.407      -5.204       2.111
x1             0.7778      0.